### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [2]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [3]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [4]:
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [5]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python', encoding='latin-1')

Explicit данные

In [6]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [7]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [8]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [9]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [10]:
user_item_csr.shape

(6041, 3953)

In [11]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [12]:
model.fit(user_item_t_csr)

100%|██████████| 100/100 [00:28<00:00,  3.54it/s, loss=0.0135]


Построим похожие фильмы по 1 movie_id = Истории игрушек

In [13]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [15]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '360    Lion King, The (1994)',
 '1838    Mulan (1998)',
 '1526    Hercules (1997)',
 '2618    Tarzan (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [16]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [17]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [18]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [19]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '2502    Matrix, The (1999)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1182    Aliens (1986)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '3402    Close Encounters of the Third Kind (1977)',
 '847    Godfather, The (1972)',
 '2460    Planet of the Apes (1968)',
 '2880    Dr. No (1962)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [8]:
from methods import SVD

svd = SVD(latent_size=32, epochs=15)
svd.fit(ratings)

100%|██████████| 1000209/1000209 [01:18<00:00, 12805.77it/s]


epoch 1: RMSE = 1.2163


100%|██████████| 1000209/1000209 [01:17<00:00, 12828.86it/s]


epoch 2: RMSE = 0.9286


100%|██████████| 1000209/1000209 [01:18<00:00, 12758.47it/s]


...

epoch 13: RMSE = 0.7410


100%|██████████| 1000209/1000209 [01:17<00:00, 12839.14it/s]


epoch 14: RMSE = 0.7345


100%|██████████| 1000209/1000209 [01:18<00:00, 12803.86it/s]


epoch 15: RMSE = 0.7290


In [9]:
from methods import Test

test = Test(movie_info, svd.U, svd.V, svd.users, svd.items, svd.U_bias, svd.V_bias)
movie, similars = test.get_similars(item_id=1)

print(f'Similars to movie: {movie}')
for _, name, cat in similars.values:
    print(f'- {name:<50} | {cat}')

Similars to movie: Toy Story (1995)
- Babe (1995)                                        | Children's|Comedy|Drama
- Jurassic Park (1993)                               | Action|Adventure|Sci-Fi
- Beauty and the Beast (1991)                        | Animation|Children's|Musical
- Princess Bride, The (1987)                         | Action|Adventure|Comedy|Romance
- Star Wars: Episode VI - Return of the Jedi (1983)  | Action|Adventure|Romance|Sci-Fi|War
- Titanic (1997)                                     | Drama|Romance
- Little Mermaid, The (1989)                         | Animation|Children's|Comedy|Musical|Romance
- Bug's Life, A (1998)                               | Animation|Children's|Comedy
- Star Wars: Episode I - The Phantom Menace (1999)   | Action|Adventure|Fantasy|Sci-Fi
- Toy Story 2 (1999)                                 | Animation|Children's|Comedy


In [10]:
user_id = 4

recs = test.get_recommendations(user_id=user_id, k=15)
print(f'Recomendations for user: {user_id}')

for _, name, cat in recs.values:
    print(f'- {name:<40} | {cat}')

Recomendations for user: 4
- Pulp Fiction (1994)                      | Crime|Drama
- Schindler's List (1993)                  | Drama|War
- Silence of the Lambs, The (1991)         | Drama|Thriller
- Godfather, The (1972)                    | Action|Crime|Drama
- 12 Angry Men (1957)                      | Drama
- To Kill a Mockingbird (1962)             | Drama
- Godfather: Part II, The (1974)           | Action|Crime|Drama
- Great Escape, The (1963)                 | Adventure|War
- Sanjuro (1962)                           | Action|Adventure
- Mister Roberts (1955)                    | Comedy|Drama|War
- Grapes of Wrath, The (1940)              | Drama
- Stalag 17 (1953)                         | Drama|War
- Animal House (1978)                      | Comedy
- Gladiator (2000)                         | Action|Drama
- Blazing Saddles (1974)                   | Comedy|Western


### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [25]:
from methods import ALS

als = ALS(latent_size=64, epochs=30, lambd=1e-4)
als.fit(ratings)

epoch 1: RMSE = 0.7278
epoch 2: RMSE = 0.6550
epoch 3: RMSE = 0.6448
epoch 4: RMSE = 0.6416
epoch 5: RMSE = 0.6403
epoch 6: RMSE = 0.6396
epoch 7: RMSE = 0.6392
epoch 8: RMSE = 0.6389
epoch 9: RMSE = 0.6387
epoch 10: RMSE = 0.6386
epoch 11: RMSE = 0.6385
epoch 12: RMSE = 0.6384
epoch 13: RMSE = 0.6384
epoch 14: RMSE = 0.6383
epoch 15: RMSE = 0.6383
epoch 16: RMSE = 0.6383
epoch 17: RMSE = 0.6383
epoch 18: RMSE = 0.6383
epoch 19: RMSE = 0.6382
epoch 20: RMSE = 0.6382
epoch 21: RMSE = 0.6382
epoch 22: RMSE = 0.6382
epoch 23: RMSE = 0.6382
epoch 24: RMSE = 0.6382
epoch 25: RMSE = 0.6382
epoch 26: RMSE = 0.6382
epoch 27: RMSE = 0.6382
epoch 28: RMSE = 0.6382
epoch 29: RMSE = 0.6382
epoch 30: RMSE = 0.6382


In [26]:
from methods import Test

test = Test(movie_info, als.U, als.V, als.users, als.items)
movie, similars = test.get_similars(item_id=1)

print(f'Similars to movie: {movie}')
for _, name, cat in similars.values:
    print(f'- {name:<60} | {cat}')

Similars to movie: Toy Story (1995)
- Babe (1995)                                                  | Children's|Comedy|Drama
- Forrest Gump (1994)                                          | Comedy|Romance|War
- Silence of the Lambs, The (1991)                             | Drama|Thriller
- Princess Bride, The (1987)                                   | Action|Adventure|Comedy|Romance
- Stand by Me (1986)                                           | Adventure|Comedy|Drama
- Groundhog Day (1993)                                         | Comedy|Romance
- Back to the Future (1985)                                    | Comedy|Sci-Fi
- Bug's Life, A (1998)                                         | Animation|Children's|Comedy
- American Beauty (1999)                                       | Comedy|Drama
- Toy Story 2 (1999)                                           | Animation|Children's|Comedy


In [29]:
user_id = 4

recs = test.get_recommendations(user_id=user_id, k=15)
print(f'Recomendations for user: {user_id}')

for _, name, cat in recs.values:
    print(f'- {name:<50} | {cat}')

Recomendations for user: 4
- Star Wars: Episode IV - A New Hope (1977)          | Action|Adventure|Fantasy|Sci-Fi
- Terminator 2: Judgment Day (1991)                  | Action|Sci-Fi|Thriller
- Die Hard (1988)                                    | Action|Thriller
- E.T. the Extra-Terrestrial (1982)                  | Children's|Drama|Fantasy|Sci-Fi
- Raiders of the Lost Ark (1981)                     | Action|Adventure
- Good, The Bad and The Ugly, The (1966)             | Action|Western
- Alien (1979)                                       | Action|Horror|Sci-Fi|Thriller
- Terminator, The (1984)                             | Action|Sci-Fi|Thriller
- Indiana Jones and the Last Crusade (1989)          | Action|Adventure
- Butch Cassidy and the Sundance Kid (1969)          | Action|Comedy|Western
- Jaws (1975)                                        | Action|Horror
- Rocky (1976)                                       | Action|Drama
- Saving Private Ryan (1998)                         | Acti

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [55]:
from methods import BPR

bpr = BPR(learning_rate=1e-1, lambd=1e-7, epochs=5)
bpr.fit(ratings)

 20%|██        | 1/5 [00:26<01:45, 26.39s/it]

epoch 1: AUC = 0.8835


 40%|████      | 2/5 [00:52<01:19, 26.34s/it]

epoch 2: AUC = 0.9094


 60%|██████    | 3/5 [01:18<00:52, 26.30s/it]

epoch 3: AUC = 0.9314


 80%|████████  | 4/5 [01:45<00:26, 26.28s/it]

epoch 4: AUC = 0.9427


100%|██████████| 5/5 [02:11<00:00, 26.34s/it]

epoch 5: AUC = 0.9459


In [56]:
from methods import Test

test = Test(movie_info, bpr.U, bpr.V, bpr.users, bpr.items)
movie, similars = test.get_similars(item_id=1)

print(f'Similars to movie: {movie}')
for _, name, cat in similars.values:
    print(f'- {name:<60} | {cat}')

Similars to movie: Toy Story (1995)
- Babe (1995)                                                  | Children's|Comedy|Drama
- Shawshank Redemption, The (1994)                             | Drama
- Forrest Gump (1994)                                          | Comedy|Romance|War
- Aladdin (1992)                                               | Animation|Children's|Comedy|Musical
- Silence of the Lambs, The (1991)                             | Drama|Thriller
- Casablanca (1942)                                            | Drama|Romance|War
- Bug's Life, A (1998)                                         | Animation|Children's|Comedy
- Shakespeare in Love (1998)                                   | Comedy|Romance
- American Beauty (1999)                                       | Comedy|Drama
- Toy Story 2 (1999)                                           | Animation|Children's|Comedy


In [57]:
user_id = 4

recs = test.get_recommendations(user_id=user_id, k=15)
print(f'Recomendations for user: {user_id}')

for _, name, cat in recs.values:
    print(f'- {name:<70} | {cat}')

Recomendations for user: 4
- Toy Story (1995)                                                       | Animation|Children's|Comedy
- Braveheart (1995)                                                      | Action|Drama|War
- Star Wars: Episode IV - A New Hope (1977)                              | Action|Adventure|Fantasy|Sci-Fi
- Shawshank Redemption, The (1994)                                       | Drama
- Fugitive, The (1993)                                                   | Action|Thriller
- Terminator 2: Judgment Day (1991)                                      | Action|Sci-Fi|Thriller
- Silence of the Lambs, The (1991)                                       | Drama|Thriller
- Godfather, The (1972)                                                  | Action|Crime|Drama
- Star Wars: Episode V - The Empire Strikes Back (1980)                  | Action|Adventure|Drama|Sci-Fi|War
- Princess Bride, The (1987)                                             | Action|Adventure|Comedy|Romance
-

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных